# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [44]:
#!pip install chromadb

In [45]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [46]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [47]:
llm = OpenAI(temperature=0)

In [48]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [49]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [50]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [51]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [52]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [66]:
docs = loader.load()  # loading page contents into documents
ruff_texts = text_splitter.split_documents(docs)  # Split the documents into smaller chunks
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")  # Create a vector store
ruff = RetrievalQA.from_chain_type(
    llm=llm,  # OpenAI language model
    chain_type="stuff",  # The type of chain to use, stuff means the text is stored as is, without summarizing or any other processing
    retriever=ruff_db.as_retriever()  # The retriever to use
)

Created a chunk of size 2130, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


In [57]:
ruff_texts

[Document(metadata={'source': 'https://beta.ruff.rs/docs/faq/', 'title': 'FAQ | Ruff', 'description': 'An extremely fast Python linter and code formatter, written in Rust.', 'language': 'en'}, page_content='FAQ | Ruff\n\n          Skip to content\n        \n\n            Ruff\n          \n\n            \n              FAQ\n            \n          \n\n\n            Initializing search\n          \n\n\n    ruff\n  \n\n    Ruff\n  \n\n\n    ruff\n  \n\n\n    \n  \n    Overview\n  \n\n    \n  \n\n    \n  \n    Tutorial\n  \n\n    \n  \n\n    \n  \n    Installing Ruff\n  \n\n    \n  \n\n    \n  \n    The Ruff Linter\n  \n\n    \n  \n\n    \n  \n    The Ruff Formatter\n  \n\n    \n  \n\n\n    \n  \n    Editors\n  \n\n    \n  \n\n            \n  \n    Editors\n  \n\n          \n\n\n    \n  \n    Editor Integration\n  \n\n    \n  \n\n    \n  \n    Setup\n  \n\n    \n  \n\n    \n  \n    Features\n  \n\n    \n  \n\n    \n  \n    Settings\n  \n\n    \n  \n\n    \n  \n    Migrating from ruff-lsp\n

## Create the Agent

In [59]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent

In [61]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.invoke,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.invoke,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [62]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

C:\Users\Nath\AppData\Local\Temp\ipykernel_23256\1834837320.py:3: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [63]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: {'query': 'What did Biden say about Ketanji Brown Jackson in the state of the union address?', 'result': "\nBiden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}
Thought: I now know the final answer.
Final Answer: Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.

> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [67]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: {'query': 'Why use ruff over flake8?', 'result': ' Ruff offers a larger rule set and can automatically fix its own lint violations, while also being able to be used as a formatter. It also supports linting for any Python version from 3.7 onwards, including Python 3.13. Additionally, Ruff does not require the installation of Rust, making it more accessible for users.'}
Thought: This information is helpful, but I should also check the State of Union QA System to see if there are any other reasons mentioned.
Action: State of Union QA System
Action Input: "Why use ruff over flake8?"
Observation: {'query': 'Why use ruff over flake8?', 'result': " I don't know."}
Thought: It seems like the State of Union QA System does not have 

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers a larger rule set, automatic fixing of lint violations, and support for multiple Python versions, making it a more comprehensive and accessible linter compared to flake8.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [68]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.invoke,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [69]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [70]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation: 
Biden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': "\nBiden said that he nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court, and that she is one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [71]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff and flake8 are both python linters, so they have similar functions. However, there may be specific reasons why one is preferred over the other.
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation:  Ruff offers a larger rule set and can automatically fix its own lint violations, while also being able to be used as a formatter. It also supports linting for any Python version from 3.7 onwards, including Python 3.13. Additionally, Ruff does not require the installation of Rust, making it more accessible for users.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': ' Ruff offers a larger rule set and can automatically fix its own lint violations, while also being able to be used as a formatter. It also supports linting for any Python version from 3.7 onwards, including Python 3.13. Additionally, Ruff does not require the installation of Rust, making it more accessible for users.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [73]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.invoke,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.invoke,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [74]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [75]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should use the Ruff QA System to answer this question since it is specifically about ruff and Jupyter Notebooks.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation: {'query': 'What tool does ruff use to run over Jupyter Notebooks?', 'result': ' Ruff does not currently support Jupyter Notebooks.'}
Thought: Since Ruff does not currently support Jupyter Notebooks, I should use the State of Union QA System to check if the president mentioned it.
Action: State of Union QA System
Action Input: Did the president mention Jupyter Notebooks in the state of the union?
Observation: {'query': 'Did the president mention Jupyter Notebooks in the state of the union?', 'result': ' No, the president did not mention Jupyter Notebooks in the state of the union.'}
Thought: I now know the final answer.
Final Answer: No, the president did not mention Jupyter Notebooks in the state of the union.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'No, the president did not mention Jupyter Notebooks in the state of the union.'}

## Let's try the same process with a different dataset

Dataset:    [Romeo And Juliet](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/romeoandjuliet.txt)

In [81]:
# create llm
llm = OpenAI(temperature=0)

# load dataset
doc_path = str(Path(*relevant_parts) / "romeoandjuliet.txt")

# store dataset in documents
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# store documents in chroma
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="romeo-and-juliet")

# create retriever
romeo_and_juliet = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

# create a webloader with Shakespeare QnA
loader = WebBaseLoader("https://parade.com/1206030/marynliles/shakespeare-trivia/")

# loading webpage contents into documents
docs = loader.load()
shakespeare_qna_texts = text_splitter.split_documents(docs)
shakespeare_qna_db = Chroma.from_documents(shakespeare_qna_texts, embeddings, collection_name="shakespeare_qna")
shakespeare_qna = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=shakespeare_qna_db.as_retriever()
)

tools = [
    Tool(
        name="Romeo and Juliet",
        func=state_of_union.invoke,
        description="useful for when you need to answer questions about Shakespeare's book Romeo and Juliet. Input should be a fully formed question.",
    ),
    Tool(
        name="Shakespeare QnA",
        func=ruff.invoke,
        description="useful for when you need to answer questions about Shakespeare, his life and his work. Input should be a fully formed question.",
    ),
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


Created a chunk of size 1018, which is longer than the specified 1000
Created a chunk of size 1479, which is longer than the specified 1000
Created a chunk of size 1901, which is longer than the specified 1000
Created a chunk of size 1044, which is longer than the specified 1000
Created a chunk of size 1015, which is longer than the specified 1000
Created a chunk of size 1381, which is longer than the specified 1000
Created a chunk of size 1082, which is longer than the specified 1000
Created a chunk of size 1468, which is longer than the specified 1000
Created a chunk of size 1415, which is longer than the specified 1000
Created a chunk of size 2240, which is longer than the specified 1000
Created a chunk of size 1430, which is longer than the specified 1000
Created a chunk of size 1541, which is longer than the specified 1000
Created a chunk of size 1006, which is longer than the specified 1000
Created a chunk of size 1370, which is longer than the specified 1000
Created a chunk of s

In [89]:
question = "Where is Romeo exiled at the end of the book Romeo and Juliet? Was Shakespeare burried in this town?"
agent.invoke(question)



> Entering new AgentExecutor chain...
 I should use the Romeo and Juliet tool to find out where Romeo was exiled.
Action: Romeo and Juliet
Action Input: Where is Romeo exiled at the end of the book Romeo and Juliet?
Observation: {'query': 'Where is Romeo exiled at the end of the book Romeo and Juliet?', 'result': ' Romeo is exiled to Mantua at the end of the book Romeo and Juliet.'}
Thought: Now I need to find out if Shakespeare was buried in Mantua.
Action: Shakespeare QnA
Action Input: Was Shakespeare buried in Mantua?
Observation: {'query': 'Was Shakespeare buried in Mantua?', 'result': ' No, Shakespeare was buried in Stratford-upon-Avon.'}
Thought: I now know the final answer.
Final Answer: No, Shakespeare was not buried in Mantua.

> Finished chain.


{'input': 'Where is Romeo exiled at the end of the book Romeo and Juliet? Was Shakespeare burried in this town?',
 'output': 'No, Shakespeare was not buried in Mantua.'}